In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
# Import the library to create the model
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


class DenoisingModel(nn.Module):
    def __init__(self):
        super(DenoisingModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 28 * 28, 100)
        self.fc2 = nn.Linear(100, 28 * 28)

    def forward(self, x, t):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
def add_noise(imgs, noise_level):
    return imgs + noise_level * torch.randn_like(imgs)

def train(model, dataloader, optimizer, num_steps=10):
    model.train()
    for step in range(num_steps):
        for img, _ in dataloader:
            noise_level = (step + 1) / num_steps
            noisy_imgs = add_noise(img, noise_level)
            optimizer.zero_grad()
            predicted_noise = model(noisy_imgs, step)
            loss = nn.functional.mse_loss(predicted_noise, noisy_imgs - img)
            loss.backward()
            optimizer.step()


model = DenoisingModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Train the model
train(model, dataloader, optimizer)
